In [1]:
from typing import List, Dict, Tuple
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline
from torchvision import datasets
import torchvision.transforms as transforms
import numpy as np
dev = 'cpu'
if torch.cuda.is_available():
    dev = 'cuda:0'
print("Running on:",dev)
device = torch.device(dev)

Running on: cuda:0


# Quivers and quiver representations

The quiver class is designed to define an acyclic quiver with no double edges. The initial quiver is not required to have a bias vertex; such a vertex can be added with the ```add_bias()``` method.

The input list of vertices is meant to be a list of strings, one for each vertex. It is best to avoid the label ```bias``` among the vertices. 

The input list of edges is meant to be a tuple ```e = (e[0], e[1])``` where ```e[0]``` is the source and ```e[1]``` is the target. No double edges or loops are allowed.

In [2]:
class quiver:
    """Quiver class. Vertices are given as a list of strings. Edges are given as a list of pairs.
    
    Attributes and Methods.
    """
    def __init__(self, vertices : List[str], edges : List[Tuple[str]]):
        self.vertices = vertices
        # Add assert to check no repeated vertices
        # E.g. assert len(set(vertices)) == len(vertices)
        # Assert there is no bias initially, or if there is, it has the desired properties
        
        self.edges = edges
        # Add assert to check that edges is a list of pairs
        # First entry of the pair is the sourse, second is the target
        # Source and target of each edge should be in the vertex set      
        # Separate class for edges? vertices?
        
        # Get the sources and sinks
        sources = set(self.vertices)
        sinks = set(self.vertices)
        for e in self.edges:
            sources.discard(e[1])
            sinks.discard(e[0])
        self.sources = sources
        self.sinks = sinks
    
    # Check that the quiver is acyclic
    def check_acyclic(self):
        None
        # One way: find all sources, do depth-first search
        
    # Check that the vertices are in topological order    
    def check_top_order(self):
        indices = {}
        for i,v in enumerate(self.vertices):
            indices[v] = i
        return all([indices[e[0]] <  indices[e[1]] for e in self.edges])
        
    # Get the incoming edges for every vertex
    def get_incoming(self, vertex):
        assert vertex in self.vertices, "No such vertex found"
        return [e for e in self.edges if e[1] == vertex]
        # Can get the incoming neighbors as [e[1] for e in self.get_incoming(vertex)]
        
    # Add a bias vertex. Considering alternatives to this ... 
    def add_bias(self):
        # Add bias vertex. This will not disturb the topological order.
        for v in self.vertices:
            if v not in self.sources:
                self.edges.append(('bias', v))
        self.vertices = ['bias'] + self.vertices
        return
    
    #########
    # Check if a vertex is a sink (Don't really need this any more)
    def is_sink(self, vertex):
        assert vertex in self.vertices, "No such vertex found"
        return all([e[0] != vertex for e in self.edges])
    


In [3]:
class quiver_rep:
    """Quiver representation class. Input a quiver with dimension vector and a matrix for each edge."""
    def __init__(self, quiver: quiver, dims: Dict[str,int], matrices: Dict[str, np.array]):
        self.quiver = quiver
        self.dims = dims
        self.matrices = matrices
        
        # Check the dimension vector
        assert len(dims) == len(self.quiver.vertices), "Inappropriate dimension vector"
        for v in dims:
            assert v in self.quiver.vertices, "Inappropriate dim vector"
            assert isinstance(dims[v], int) and dims[v] >=0, "Dimension needs to be a positive integer"
        
        if 'bias' in dims:
            assert dims['bias'] == 1, "Dimension at bias needs to be 1"

            
        # Check the matrices
        assert len(matrices) == len(self.quiver.edges), "Matrices error"
        for e in matrices:
            assert e in self.quiver.edges, "Matrices error"
            assert isinstance(matrices[e], np.ndarray), "Matrices error" # May need fixing
            assert np.shape(matrices[e]) == (dims[e[1]], dims[e[0]]), "Dimension error"
            
            
    # Compute the reduced dimension vector
    def comp_dims_red(self) -> Dict:
        
        assert self.quiver.check_top_order(), "Order of the vertices is not topological"

        dims_red = {}
        for i in self.quiver.vertices:
            if i == 'bias' or i in self.quiver.sources or i in self.quiver.sinks:
                dims_red[i] = self.dims[i]
            else:
                incoming = self.quiver.get_incoming(i)
                dims_red[i] = min(self.dims[i], sum([dims_red[e[0]] for e in incoming]) )

                    
        self.dims_red = dims_red        
        return dims_red
    
    
    # Auxiliary function
    def padzeros(self, M, newrows, newcols = None):
        oldrows, oldcols = M.shape
        if newcols == None:
            newcols = oldcols
        return np.pad(M,((0,newrows-oldrows),(0,newcols-oldcols)),mode="constant")
    
    def padzeros_to_dim(self, new_dims):
        new_matrices = {}
        for e in self.matrices:
            new_matrices[e] = self.padzeros(self.matrices[e],new_dims[e[1]],new_dims[e[0]])
        return quiver_rep(self.quiver,new_dims,new_matrices)
 
    
    # QR dimensional reduction algorithm
    def QRDimRed(self, verbose : bool = False ):
        dims = self.dims
        matrices = self.matrices
        quiver = self.quiver
        vertices = quiver.vertices
        edges = quiver.edges

        # Check that vertices are in a topological order
        assert quiver.check_top_order(), "Order of the vertices is not topological"

        # Compute the reduced dimension vector
        dims_red = self.comp_dims_red()
        # print(dims, dims_red)

        # Q = dictionary mapping each vertex to an orthogonal matrix
        Q = {}

        # Vmatrices = matrices of the reduced representation V, mapping each edge to a matrix
        Vmatrices = {}

        if verbose:
            print(quiver.edges)
            print(quiver.vertices)

        for i in vertices:
            incoming = quiver.get_incoming(i)

            # Case of a source vertex
            if incoming == []:
                Q[i] = np.eye(dims[i])

            # Case of a hidden vertex    
            elif i not in quiver.sinks:

                # Compute the matrix to be QR-decomposed
                M = np.array([])
                for e in incoming:
                    # Transform weights on incoming edges
                    Qj = Q[e[0]]
                    Me = matrices[e] @ Qj[:,:dims_red[e[0]]]
                    if np.shape(M) == (0,):
                        M = Me
                    else:
                        M = np.hstack((M, Me))

                Q[i], R = np.linalg.qr(M, mode="complete")

                # Case of reduction 
                if dims_red[i] < dims[i]: 
                    R = R[:dims_red[i]]

                # Process and add to the dictionaries
                for e in incoming:                       
                    # Extract V_e from R_i for all incoming edges e
                    Vmatrices[e] = R[:,:dims_red[e[0]]]
                    R = R[:,dims_red[e[0]]:]

            # Case of a sink (no reduction)
            else:
                Q[i] = np.eye(dims[i])
                for e in incoming:
                    # Transform weights on incoming edges
                    Qj = Q[e[0]]
                    Vmatrices[e] = matrices[e] @ Qj[:,:dims_red[e[0]]]                


        # Make V into a representation
        V = quiver_rep(quiver, dims_red, Vmatrices)

        # Verify that V is a subrepresentation of Q^{-1} W  
        for e in quiver.edges:
            Qi = Q[e[0]]
            Qj = Q[e[1]]
            max_diff = np.max(np.abs(np.transpose(Qj) @ matrices[e] @ Qi[:,:dims_red[e[0]]] 
                         - self.padzeros(Vmatrices[e], dims[e[1]])))
            assert max_diff < 1e-10, "Error in the algorithm"

        return Q, V
    
    
    def reduced_representation(self, verbose : bool = False ):
        return self.QRDimRed(verbose)[1]
                
    def transformed_representation(self):
        Q, V = self.QRDimRed()
        transformed_mat_dict = {}
        for e in self.quiver.edges:
            transformed_mat_dict[e] = np.transpose(Q[e[1]]) @ self.matrices[e] @ Q[e[0]]
        
        Q_inv_W = quiver_rep(self.quiver, self.dims, transformed_mat_dict)
        
        return Q_inv_W
    
    def Q_act(self,Q):
        new_matrices = {}
        for w in self.matrices:
            new_matrices[w] = Q[w[1]] @ self.matrices[w] @ np.transpose(Q[w[0]])
        return quiver_rep(self.quiver, self.dims, new_matrices)

# Dimensional reduction algorithm

In [4]:
# Auxiliary function

def padzeros(M,newrows,newcols = None):
    oldrows, oldcols = M.shape
    if newcols == None:
        newcols = oldcols
    return np.pad(M,((0,newrows-oldrows),(0,newcols-oldcols)),mode="constant")

In [5]:
# QR dimensional reduction algorithm
# This is incorporated in the quiver_rep class

def QRDimRed(W : quiver_rep, verbose : bool = False ):
    dims = W.dims
    matrices = W.matrices
    quiver = W.quiver
    vertices = quiver.vertices
    edges = quiver.edges
    
    # Check that vertices are in a topological order
    assert quiver.check_top_order(), "Order of the vertices is not topological"
    
    # Compute the reduced dimension vector
    dims_red = W.comp_dims_red()
    # print(dims, dims_red)
    
    # Q = dictionary mapping each vertex to an orthogonal matrix
    Q = {}
    
    # Vmatrices = matrices of the reduced representation V, mapping each edge to a matrix
    Vmatrices = {}
    
    if verbose:
        print(quiver.edges)
        print(quiver.vertices)
    
    for i in vertices:
        incoming = quiver.get_incoming(i)
        
        # Case of a source vertex
        if incoming == []:
            Q[i] = np.eye(dims[i])
            
        # Case of a hidden vertex    
        elif i not in quiver.sinks:
            
            # Compute the matrix to be QR-decomposed
            M = np.array([])
            for e in incoming:
                # Transform weights on incoming edges
                Qj = Q[e[0]]
                Me = matrices[e] @ Qj[:,:dims_red[e[0]]]
                if np.shape(M) == (0,):
                    M = Me
                else:
                    M = np.hstack((M, Me))
                    
            Q[i], R = np.linalg.qr(M, mode="complete")

            # Case of reduction 
            if dims_red[i] < dims[i]: 
                R = R[:dims_red[i]]
                
            # Process and add to the dictionaries
            for e in incoming:                       
                # Extract V_e from R_i for all incoming edges e
                Vmatrices[e] = R[:,:dims_red[e[0]]]
                R = R[:,dims_red[e[0]]:]
                
        # Case of a sink (no reduction)
        else:
            Q[i] = np.eye(dims[i])
            for e in incoming:
                # Transform weights on incoming edges
                Qj = Q[e[0]]
                Vmatrices[e] = matrices[e] @ Qj[:,:dims_red[e[0]]]                

    
    # Make V into a representation
    V = quiver_rep(quiver, dims_red, Vmatrices)

    # Verify that V is a subrepresentation of Q^{-1} W  
    for e in quiver.edges:
        Qi = Q[e[0]]
        Qj = Q[e[1]]
        max_diff = np.max(np.abs(np.transpose(Qj) @ matrices[e] @ Qi[:,:dims_red[e[0]]] 
                     - padzeros(Vmatrices[e], dims[e[1]])))
        assert max_diff < 1e-10, "Error in the algorithm"

    return Q, V

# Quiver Neural Networks

In [6]:
class RadAct(nn.Module):
    def __init__(self, eta = F.relu):
        super().__init__()
        self.eta = eta
        self.shift = 0 
        # Add internal bias/shift later
        
    def forward(self,x):
        # x: [Batch x Channel]
        # Radial activations
        r = torch.linalg.norm(x, dim=-1) 
        if torch.min(r) < 1e-6:
            r += 1e-6
        scalar = self.eta(r + self.shift) / r
        return x * scalar.unsqueeze(-1)   

In [7]:
class QuiverNN(nn.Module):
    
    def __init__(self, eta: float , quiver: quiver, dims: Dict[str,int] ):
        super().__init__()
        self.eta = eta
        self.quiver = quiver
        self.dims = dims
        self.matrices = nn.ModuleDict()
        
        # Linear layer for each edge
        for e in quiver.edges:
            self.matrices[self.edge_tup_to_str(e)] = nn.Linear(self.dims[e[0]], self.dims[e[1]], bias = False)
        
        assert quiver.check_top_order(), "Vertices not in topological order."
        
        # Radial activations
        self.act = RadAct(self.eta)
        
        # Add assert statement to check that dims is a dimension vector for Q
          
    # Encode each edge as a string
    def edge_tup_to_str(self, e : Tuple[str]):
        assert e in self.quiver.edges
        (t,h) = e
        return t + " to " + h
    
    # Extract the (source, target) pair from each string encoding an edge
    def edge_str_to_tup(self, e_str : str):
        t,to,h = e_str.split()
        assert (t,h) in self.quiver.edges
        return (t,h)
    
    # Get the matrix of an edge
    def get_matrix(self, e):
        return self.matrices[self.edge_tup_to_str(e)].weight


    # The feedforward function
    def forward(self, x, non_linear = True):
        
        # Initialize Data Flow
        h = {}
        
        # Bias 
        h['bias'] = torch.tensor(1.0)
        
        # Sources
        for v in self.quiver.sources:
            h[v] = x[v]
            batch_size = x[v].shape[0]
        
        # Assert batchsize is same for all sources 
        
        for v in self.quiver.vertices:
            
            # Non-source vertices
            if v not in self.quiver.sources:
                h[v] = torch.zeros(batch_size,self.dims[v])
                for e in self.quiver.get_incoming(v):
                    e_lin = self.matrices[self.edge_tup_to_str(e)]
                    h[e[1]] += e_lin(h[e[0]])
                if non_linear:
                    h[v] = self.act(h[v])
        
        # Feedforward function output
        out = {}
        for v in self.quiver.sinks:
            out[v] = h[v]
            
        return out
            
    
    def set_weights(self, new_weights: quiver_rep):
        assert new_weights.quiver == self.quiver, "weights have different quiver"
        
        for e in self.quiver.edges:
            self.matrices[self.edge_tup_to_str(e)].weight = \
                torch.nn.Parameter(torch.tensor(new_weights.matrices[e],dtype = torch.float))
        self.dims = new_weights.dims

    def export_weights(self) -> quiver_rep:
        quiver_rep_matrix_dict = {}
        for e in self.quiver.edges:
            quiver_rep_matrix_dict[e] = self.matrices[self.edge_tup_to_str(e)].weight.detach().cpu().numpy()
        return quiver_rep(self.quiver, self.dims, quiver_rep_matrix_dict)
    
    def export_reduced_weights(self) -> quiver_rep:
        exported_rep = self.export_weights()
        return exported_rep.reduced_representation()
    
    def transformed_network(self):
        exported_rep = self.export_weights()
        rep_transformed = exported_rep.transformed_representation()
        net_trans = QuiverNN(self.eta, self.quiver, self.dims)
        net_trans.set_weights(rep_transformed)
        #net_trans.set_activation_biases(self.export_activation_biases())
        return net_trans
        
    def reduced_network(self):
        reduced_rep = self.export_reduced_weights()
        net_reduced = QuiverNN(self.eta, self.quiver, reduced_rep.dims)
        net_reduced.set_weights(reduced_rep)
        #net_trans.set_activation_biases(self.export_activation_biases())
        return net_reduced
    
    def Q_act(self,Q):
        QW = self.export_weights().Q_act(Q)
        self.set_weights(QW)
    
    # Might not need these:
    def export_activation_biases(self) -> List[float]:
        None
    def set_activation_biases(self, new_biases: List[float]):    
        None

# Example 1

In [8]:
vertex_list1 = ['a', 'b', 'c', 'd']
edge_list1 = [('a', 'b'), ('a','c'), ('b','c'), ('c', 'd')]
dim_vector1 = {'bias' : 1, 'a': 2, 'b': 4, 'c': 8, 'd': 2 }
quiv_ex1 = quiver(vertex_list1, edge_list1)
quiv_ex1.add_bias()

quiverNN1 = QuiverNN(eta = F.relu, quiver = quiv_ex1, dims = dim_vector1)

# Representation of this quiver

maps1 = {('a', 'b') : np.random.rand(4, 2), 
        ('a', 'c') : np.random.rand(8, 2), 
        ('b', 'c') : np.random.rand(8, 4),
        ('c', 'd') : np.random.rand(2, 8),
        ('bias', 'b') : np.random.rand(4, 1),
        ('bias', 'c') : np.random.rand(8, 1),
        ('bias', 'd') : np.random.rand(2, 1)}

ex_rep1 = quiver_rep(quiv_ex1, dim_vector1, maps1)
quiverNN1.set_weights(ex_rep1)

In [9]:
quiverNN1.export_weights().comp_dims_red()

{'bias': 1, 'a': 2, 'b': 3, 'c': 6, 'd': 2}

# Example 2

In [10]:
vertex_list2 = ['a', 'b', 'c', 'd', 'e']
edge_list2 = [('a', 'c'), ('b','c'), ('c','d'), ('c', 'e')]
dim_vector2 = {'bias' : 1, 'a': 1, 'b': 2, 'c': 8, 'd': 2 , 'e': 6}
quiv_ex2 = quiver(vertex_list2, edge_list2)
quiv_ex2.add_bias()

quiverNN2 = QuiverNN(eta = F.relu, quiver = quiv_ex2, dims = dim_vector2)

# Representation of this quiver

maps2 = {('a', 'c') : np.random.rand(8, 1), 
        ('b', 'c') : np.random.rand(8, 2), 
        ('c', 'd') : np.random.rand(2, 8),
        ('c', 'e') : np.random.rand(6, 8),
        ('bias', 'c') : np.random.rand(8, 1),
        ('bias', 'd') : np.random.rand(2, 1),
        ('bias', 'e') : np.random.rand(6, 1)}

ex_rep2 = quiver_rep(quiv_ex2, dim_vector2, maps2)
quiverNN2.set_weights(ex_rep2)

In [11]:
quiverNN2.export_weights().comp_dims_red()

{'bias': 1, 'a': 1, 'b': 2, 'c': 4, 'd': 2, 'e': 6}

# Example 3

In [12]:
vertex_list3 = ['a', 'b', 'c', 'd', 'e']
edge_list3 = [('a', 'b'), ('a','c'), ('b','d'), ('c', 'd'),('d','e')]

dim_vector3 = {'bias':1, 'a': 2, 'b': 4, 'c': 4, 'd': 8 , 'e': 2}
quiv_ex3 = quiver(vertex_list3, edge_list3)
quiv_ex3.add_bias()

quiverNN3 = QuiverNN(eta = F.relu, quiver = quiv_ex3, dims = dim_vector3)

# Representation of this quiver


maps3 = {('a', 'b') : np.random.rand(4, 2), 
        ('a', 'c') : np.random.rand(4, 2), 
        ('b', 'd') : np.random.rand(8, 4),
        ('c', 'd') : np.random.rand(8, 4),
        ('d', 'e') : np.random.rand(2, 8),
        ('bias','b') : np.random.rand(4, 1),
        ('bias','c') : np.random.rand(4, 1),
        ('bias','d') : np.random.rand(8, 1),
        ('bias','e') : np.random.rand(2, 1),
        }

ex_rep3 = quiver_rep(quiv_ex3, dim_vector3, maps3)

quiverNN3.set_weights(ex_rep3)

In [13]:
quiverNN3.export_weights().comp_dims_red()

{'bias': 1, 'a': 2, 'b': 3, 'c': 3, 'd': 7, 'e': 2}

# Test Theorem

### Setup

In [14]:
def single_mask(red_rows, red_cols, orig_rows, orig_cols):
    assert red_rows <= orig_rows 
    assert red_cols <= orig_cols
    mask = torch.ones((orig_rows, orig_cols))
    for j in range(red_rows, orig_rows):
        mask[j][:red_cols] = 0
    return mask

In [15]:
def gamma(model, x, y, lr, verbose = False):
    
    y_hat = model(x)

    loss = torch.tensor(0.0)
    for k in y:
        loss += (y_hat[k] - y[k]).pow(2).sum()
    loss.backward()
    
    if verbose:
        print(loss.item())
    
    with torch.no_grad(): 
        for e in model.quiver.edges:
            p = model.get_matrix(e)
            p -= lr * p.grad
    
    return model


In [16]:
def gammaPGD(model, x, y, lr, verbose = False):
    
    y_hat = model(x)
    
    dims = model.dims 
    red_dims = model.export_reduced_weights().dims
    
    loss = torch.tensor(0.0)
    for k in y:
        loss += (y_hat[k] - y[k]).pow(2).sum()
    loss.backward()
    
    if verbose:
        print(loss.item())
    
    with torch.no_grad(): 
        for e in model.quiver.edges:
            p = model.get_matrix(e)
            mask_grad = single_mask(red_dims[e[1]],red_dims[e[0]],dims[e[1]],dims[e[0]])
            p -= lr * (mask_grad * p.grad)
    
    return model
    

### Test Function

In [17]:
def test_theorem(quiverNN, x, y):
    
    # Define Weight Matrix Tuples.
    W = quiverNN.export_weights()
    T_net = quiverNN.transformed_network()
    Q,V = quiverNN.export_weights().QRDimRed()  # V = W_red
    T = T_net.export_weights()
    red_quiverNN = quiverNN.reduced_network()

    # Duplicate Networks
    gd_quiverNN = type(quiverNN)(quiverNN.eta, quiverNN.quiver, quiverNN.dims) # get a new instance
    gd_quiverNN.load_state_dict(quiverNN.state_dict())

    pgd_quiverNN = type(quiverNN)(quiverNN.eta, quiverNN.quiver, quiverNN.dims) # get a new instance
    pgd_quiverNN.load_state_dict(quiverNN.state_dict())

    gd_red_quiverNN = type(red_quiverNN)(red_quiverNN.eta, red_quiverNN.quiver, red_quiverNN.dims) # get a new instance
    gd_red_quiverNN.load_state_dict(red_quiverNN.state_dict())

    pgd_T_quiverNN = type(T_net)(T_net.eta, T_net.quiver, T_net.dims) # get a new instance
    pgd_T_quiverNN.load_state_dict(T_net.state_dict())

    gd_T_quiverNN = type(T_net)(T_net.eta, T_net.quiver, T_net.dims) # get a new instance
    gd_T_quiverNN.load_state_dict(T_net.state_dict())
    
    # Test Theorem 4.1
    yhat = quiverNN(x)
    yredhat = red_quiverNN(x)

    diff1 = 0.0
    for k in yhat:
        diff1 += (torch.sum(yhat[k] - yredhat[k])).detach().item()

    # Test Theorem 5.1
    
    # Gamma(W) = Q Gamma(T)

    lr = 0.001
    
    gamma(gd_quiverNN, x, y, lr)
    gammaW = gd_quiverNN.export_weights()

    gamma(gd_T_quiverNN, x, y, lr)
    gammaT = gd_T_quiverNN.export_weights()

    QgammaT = gammaT.Q_act(Q)

    diff2 = 0.0
    for k in W.matrices:
        diff2 += np.sum(np.abs(QgammaT.matrices[k] - gammaW.matrices[k]))

    # Gamma_Proj(T) - T =( Gamma(V) -V )
    gamma(gd_red_quiverNN, x, y, lr, False);
    gammaPGD(pgd_T_quiverNN, x, y, lr, False)
    pgd_T = pgd_T_quiverNN.export_weights()

    gd_V_pad = gd_red_quiverNN.export_weights().padzeros_to_dim(T.dims)
    V_pad = V.padzeros_to_dim(T.dims)

    diff3 = 0.0
    for k in T.matrices:
        diff3 += np.sum(pgd_T.matrices[k] - T.matrices[k]-(gd_V_pad.matrices[k] - V_pad.matrices[k]))
    
    return np.array([diff1,diff2,diff3])

### Run Tests

In [18]:
diffs = np.array([0.0,0.0,0.0])
for i in range(10):
    x1 = {'a': torch.rand(1,2)}
    y1 = {'d': torch.rand(1,2)}
    diffs += test_theorem(quiverNN1,x1,y1)
    
print(diffs / 10.0)

[-2.44379044e-07  1.29441872e-06 -8.22363436e-08]


In [19]:
diffs = np.array([0.0,0.0,0.0])
for i in range(10):
    x2 = {'a': torch.rand(1,1),'b': torch.rand(1,2)}
    y2 = {'d': torch.rand(1,2), 'e': torch.rand(1,6)}
    diffs += test_theorem(quiverNN2,x2,y2)
    
print(diffs / 10.0)

[-1.78813934e-07  1.61831697e-06 -1.20757976e-07]


In [20]:
diffs = np.array([0.0,0.0,0.0])
for i in range(10):
    x3 = {'a': torch.rand(1,2)}
    y3 = {'e': torch.rand(1,2)}
    diffs += test_theorem(quiverNN3,x3,y3)
    
print(diffs / 10.0)

[ 9.53674316e-08  1.92555871e-06 -2.39498373e-07]


To Do:

- Add batching for different inputs
- Generate input shape automatically
- Simplify networks outputs
- Format Outputs